Импорт библиотек

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
import warnings
from catboost import CatBoostRegressor, cv, Pool, CatBoostClassifier
import numpy as np
import sys
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split

Считывание необходимых файлов

In [3]:
df = pd.read_csv('train_dataset_train.csv',sep=',')
df.head()

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017


In [4]:
print(df.shape[0])

1091021


Посмотрим общее описание

In [5]:
df.describe(include='O')

,ticket_id,ticket_type_nm,entrance_nm,station_nm,line_nm,pass_dttm
count,1091021,1091021,1091021,1091021,1091021,1091021
unique,335533,60,426,245,16,297040
top,7992E92F9AE0F7506BD439547FD7E11F,КОШЕЛЕК,Щёлковская ( Северный ),Щёлковская,Таганско-Краснопресненская,2022-09-12 08:20:09
freq,8,262499,17810,22133,156031,29


id билета разный для всех пользователей. Следовательно, он не применим не только для задачи регрессии, но и для задачи классификации. Следовательно, в дальнейшем удалим его.

Удалим дубликаты по id билета и дате

In [6]:
df.drop_duplicates(subset=['ticket_id', 'pass_dttm'],inplace=True)
print(df.shape[0])

1091021


Посмотрим, сколько nan значений

In [7]:
df.isnull().sum()

id                0
ticket_id         0
ticket_type_nm    0
entrance_id       0
entrance_nm       0
station_id        0
station_nm        0
line_id           0
line_nm           0
pass_dttm         0
time_to_under     0
label             0
dtype: int64

Удалим nan значения

In [9]:
df.dropna(inplace=True)

Избавляемся от дублирующих по смыслу колонок, а именно id станции, входа в станцию и ветки. Также id билета, тк это бесполезный параметр(тк id для каждого билета свой)

In [10]:
df.drop(columns=['ticket_id', 'entrance_id', 'station_id','line_id'],inplace=True)
df.pass_dttm = pd.to_datetime(df.pass_dttm)

In [11]:
df.head()

,id,ticket_type_nm,entrance_nm,station_nm,line_nm,pass_dttm,time_to_under,label
0,1,Пропуск FacePay,Лефортово БКЛ,Лефортово,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,СК учащегося 30 дней,Войковская ( Южный ),Войковская,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,БСК дружинника г.Москвы,Войковская ( Южный ),Войковская,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,30 дней,Войковская ( Южный ),Войковская,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,КОШЕЛЕК,Войковская ( Южный ),Войковская,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017


Далее введём в рассмотрение 2 модели: регрессии - для предсказывания времени(time_to_under) и классификации - для предсказывания станции захода(label). 

Обе модели будут созданы на основе моделей CatBoost.

Для начала рассмотрим модель регрессии

In [16]:
class CustomCatBoostRegressor(CatBoostRegressor):
    def __init__(self, iterations=1000, cv_num=2):
        super(CatBoostRegressor, self).__init__()
        '''
        iterations: число деревьев
        '''
        self.y = None
        self.x = None
        self.iterations = iterations
        self.cv_num = cv_num
    # кросс-валидация для определения оптимального числа деревьев
    def launch_cv(self, metric='MAPE'):
        cv_data = cv(params={'loss_function': metric, 'iterations': self.iterations, 'random_seed': 0, 'depth': 10},
                     pool=Pool(self.x, label=self.y,
                     cat_features=['ticket_type_nm', 'station_nm', 'line_nm', 'entrance_nm']),
                     fold_count=self.cv_num, inverted=False, shuffle=True, partition_random_seed=0, stratified=False)
        self.set_params(iterations=np.argmin(cv_data['test-'+metric+'-mean']))
    
    # улучшенный fit, деревьев можно задать много, но выбираем лучшее количество - не переобучимся
    def fit(self, X, y=None, cat_features=None, text_features=None, embedding_features=None,
            sample_weight=None, baseline=None, use_best_model=None,
            eval_set=None, verbose=None, logging_level=None, plot=False, plot_file=None, column_description=None,
            verbose_eval=None, metric_period=None, silent=None, early_stopping_rounds=None,
            save_snapshot=None, snapshot_file=None, snapshot_interval=None, init_model=None, callbacks=None,
            log_cout=sys.stdout, log_cerr=sys.stderr):

        self.x = X
        self.y = y
        if self.cv_num<=1:
           self.set_params(iterations=self.iterations)
        self.set_params(depth=10)
        super().fit(self.x, self.y, cat_features=['ticket_type_nm', 'station_nm', 'line_nm', 'entrance_nm'])



In [18]:
# Регрессия
# метод подготовки данных: удаление столбцов и строк с nan значениями при их наличии
def preprocess_regress(df):
    bad_columns = ['time_to_under', 'id','label','ticket_id', 'entrance_id', 'station_id','line_id']
    df = df.drop(columns=[x for x in bad_columns if x in df.columns])
    return df

# метод получения временных признаков
def get_time_features(df):
    df.pass_dttm = pd.to_datetime(df.pass_dttm)
    df['day'] = df.pass_dttm.dt.dayofweek #день недели
    df['hour'] = df.pass_dttm.dt.hour # час в формате 24
    # разбиваем на промежутки активности пользования метро в течение дня, 
    df['shift'] = df['hour'].apply(lambda x: 0 if 10 <= x <= 17 else (
        1 if 0 <= x <= 6 else (2 if 7 <= x <= 9 else (3 if x >= 18 else x))))
    df['workday'] = df['day'].apply(lambda x: 0 if x == 5 or x == 6 else 1)

    df = df.drop(columns=['pass_dttm'])
    return df

df.dropna(inplace=True)
df = df.loc[df.time_to_under > 0]

x_regress_train, x_regress_test, y_regress_train, y_regress_test = train_test_split(df.drop(columns=['time_to_under']), df[['time_to_under']], test_size=0.3)

pipe_regression = Pipeline(steps=[('preprocess',FunctionTransformer(preprocess_regress)),
                                  ('time_features',FunctionTransformer(get_time_features)),
                                  ('model', CustomCatBoostRegressor(4000,2))])
pipe_regression.fit(x_regress_train, y_regress_train)
forecast_regress = pipe_regression.predict(x_regress_test)


Выходные данные были обрезаны до нескольких последних строк (5000).
3004:	learn: 0.4405758	test: 0.4497181	best: 0.4497181 (3004)	total: 24m 32s	remaining: 8m 7s
3005:	learn: 0.4405754	test: 0.4497177	best: 0.4497177 (3005)	total: 24m 32s	remaining: 8m 7s
3006:	learn: 0.4405753	test: 0.4497176	best: 0.4497176 (3006)	total: 24m 33s	remaining: 8m 6s
3007:	learn: 0.4405752	test: 0.4497175	best: 0.4497175 (3007)	total: 24m 33s	remaining: 8m 5s
3008:	learn: 0.4405730	test: 0.4497157	best: 0.4497157 (3008)	total: 24m 33s	remaining: 8m 5s
3009:	learn: 0.4405727	test: 0.4497153	best: 0.4497153 (3009)	total: 24m 34s	remaining: 8m 4s
3010:	learn: 0.4405660	test: 0.4497119	best: 0.4497119 (3010)	total: 24m 34s	remaining: 8m 4s
3011:	learn: 0.4405659	test: 0.4497118	best: 0.4497118 (3011)	total: 24m 34s	remaining: 8m 3s
3012:	learn: 0.4405656	test: 0.4497115	best: 0.4497115 (3012)	total: 24m 34s	remaining: 8m 3s
3013:	learn: 0.4405618	test: 0.4497089	best: 0.4497089 (3013)	total: 24m 35s	remaining

Затем рассмотрим модель классификации

In [19]:
class CustomCatBoostClassifier(CatBoostClassifier):
    def __init__(self, iterations=1000):
        '''
        iterations: число деревьев
        '''
        super().__init__()
        self.y = None
        self.x = None
        self.iterations = iterations 
    
    # метод создающий данные для валидации
    def set_eval_data(self):
        df = self.x.join(self.y)
        df = df.drop_duplicates(subset=['label'])
        return df.drop(columns=['label']), df[['label']]
    
    # улучшенный fit, деревьев можно задать много, но выбираем лучшее количество - не переобучимся
    def fit(self, X, y=None, cat_features=None, text_features=None, embedding_features=None, sample_weight=None,
            baseline=None, use_best_model=None,
            eval_set=None, verbose=None, logging_level=None, plot=False, plot_file=None, column_description=None,
            verbose_eval=None, metric_period=None, silent=None, early_stopping_rounds=None,
            save_snapshot=None, snapshot_file=None, snapshot_interval=None, init_model=None, callbacks=None,
            log_cout=sys.stdout, log_cerr=sys.stderr):

        self.set_params(iterations=self.iterations, loss_function='MultiClass')
        self.x = X
        self.y = y
        self.set_params(depth=10)
        eval_x, eval_y = self.set_eval_data()
        super().fit(self.x, self.y,
                    cat_features=['ticket_type_nm','station_nm', 'line_nm', 'entrance_nm'],
                    eval_set=(eval_x, eval_y),
                    use_best_model=True)


In [32]:
# Классификация
# метод подготовки данных: удаление столбцов и строк с nan значениями при их наличии
def preprocess_multiclass(df):
    bad_columns = ['time_to_under', 'pass_dttm','id', 'ticket_id', 'entrance_id', 'station_id', 'line_id']

    df = df.drop(columns=[x for x in bad_columns if x in df.columns])
    return df

x_train_class = df.drop(columns=['label'])[:600]
y_train_class = df[['label']][:600]

#тут я сам делаю тестовую выборку с учётом ограниченности тренировочной выборки
# чтоб не было случая: обучил на студ билетах, а пытаюсь сделать прогноз на подорожниках
test_class = df.loc[df.ticket_type_nm.isin(x_train_class.ticket_type_nm)&
                    df.entrance_nm.isin(x_train_class.entrance_nm) &
                    df.station_nm.isin(x_train_class.station_nm) &
                    df.line_nm.isin(x_train_class.line_nm) ]

x_test_class = test_class.drop(columns=['label'])[:180]
y_test_class = test_class[['label']][:180]

pipe_multiclass = Pipeline(steps=[('preprocess', FunctionTransformer(preprocess_multiclass)),
                              ('model', CustomCatBoostClassifier(100))])
pipe_multiclass.fit(x_train_class, y_train_class)
forecast_class = pipe_multiclass.predict(x_test_class)

Learning rate set to 0.257567
0:	learn: 5.2216947	test: 5.2952404	best: 5.2952404 (0)	total: 23.4s	remaining: 38m 33s
1:	learn: 5.1296035	test: 5.2647250	best: 5.2647250 (1)	total: 1m 45s	remaining: 1h 26m 20s
2:	learn: 5.0371899	test: 5.2307231	best: 5.2307231 (2)	total: 3m 8s	remaining: 1h 41m 43s
3:	learn: 4.9423994	test: 5.2137492	best: 5.2137492 (3)	total: 4m 30s	remaining: 1h 48m 14s
4:	learn: 4.8521917	test: 5.1794140	best: 5.1794140 (4)	total: 5m 53s	remaining: 1h 51m 50s
5:	learn: 4.7557394	test: 5.1662335	best: 5.1662335 (5)	total: 7m 14s	remaining: 1h 53m 33s
6:	learn: 4.6705747	test: 5.1538507	best: 5.1538507 (6)	total: 8m 37s	remaining: 1h 54m 37s
7:	learn: 4.5793916	test: 5.1372213	best: 5.1372213 (7)	total: 9m 58s	remaining: 1h 54m 46s
8:	learn: 4.4953028	test: 5.1108722	best: 5.1108722 (8)	total: 11m 21s	remaining: 1h 54m 51s
9:	learn: 4.4498601	test: 5.1275919	best: 5.1108722 (8)	total: 11m 22s	remaining: 1h 42m 18s
10:	learn: 4.3664379	test: 5.0975073	best: 5.0975073 

Рассчитаем метрики

In [33]:
from sklearn.metrics import r2_score, recall_score, mean_absolute_percentage_error

def result(actual_class, forecast_class, actual_regress, forecast_regress):
    print('recall: ',recall_score(actual_class, forecast_class, average='micro'))
    print('R2: ',r2_score(actual_regress, forecast_regress))
    return 0.5 * r2_score(actual_regress, forecast_regress) + 0.5 * recall_score(actual_class, forecast_class, average='micro')


In [34]:
final = result(y_test_class, forecast_class, y_regress_test, forecast_regress)
print('result: ', final)

recall:  0.18333333333333332
R2:  0.5389430631615189
result:  0.3611381982474261
